# DynamicMF 

From resource usage statistics (file **./Data/Zdat.p**), produce the resulting factorizations and store them in **./Data/Results/**

In [ ]:
# import necessary libraries
import tensorflow as tf
import numpy as np
import pickle

tf.set_random_seed(1) # set random seed

In [ ]:
# model parameters
max_iter = 5000 # maximum iterations

K_list= [3, 5, 10] # the list of K values that we use in this experiment

In [ ]:
Z_dat = pickle.load(open('./Data/Zdat.p', 'rb')) # T x N x M
N = np.shape(Z_dat)[1] # number of nodes in cluster
M = np.shape(Z_dat)[2]   # number of statistics that measure cluster performance
T = np.shape(Z_dat)[0] # number of time steps

In [ ]:
for K in K_list:# range(min_K,max_K + 1):
    print(K)
    # variables of the models
    U_bar = tf.get_variable(name = "U_bar_" + str(K), shape = [N, K], dtype = tf.float64)
    U_hat = tf.get_variable(name = "U_hat_" + str(K), shape = [T, N, K], dtype = tf.float64)
    V = tf.get_variable(name = "V_" + str(K), shape = [M, K], dtype = tf.float64)

    # data of the models
    Z = tf.placeholder(dtype = tf.float64, shape = [T, N, M], name = "Z_" + str(K))

    # computation
    temp = U_bar * U_hat
    V_t = tf.transpose(V)
    temp2 = tf.tile(input = V_t, multiples=[T, 1])
    temp2 = tf.reshape(temp2, shape = [T, K, M])
    Z_approx = tf.matmul(temp, temp2)
    
    # loss function
    loss = tf.reduce_sum(tf.square(Z_approx - Z))
    # optimizer
    optimizer = tf.train.AdamOptimizer()
    
    train = optimizer.minimize(loss)

    # training loop
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init) # reset values to wrong
    error_val = []
    for i in range(max_iter):
        train_val, loss_val, U_bar_res, U_hat_res, V_res = sess.run([train, loss, U_bar, U_hat, V], {Z: Z_dat})
        if (i % 100 == 0):
            print('iter ' + str(i) + ':' + str(loss_val))
        error_val.append(loss_val)
        
    ##### write reconstruction error in each iteration to text file #####
    with open('./Data/log/error_' + str(K) + '.txt', 'w') as out:
        for error in error_val:
            out.write("%s\n" % error)
            
    #### store result into result folder ####
    pickle.dump([V_res, U_bar_res, U_hat_res], open('./Data/Result/dynMF_' + str(K) + '.p', 'wb'))